In [ ]:
import pandas as pd
import json
from platform import python_version

: 

In [ ]:
print(python_version())
print(pd.__version__)


: 

In [ ]:
import os

: 

In [ ]:
current_directory = os.getcwd()

: 

In [ ]:
files_and_directories = os.listdir('/home/rodrigo/repos/DATAVIZ_project/')

: 

In [ ]:
print("Current Directory:", current_directory)
print("files and directories:", files_and_directories)

: 

In [ ]:
files_and_directories = os.listdir('/home/rodrigo/repos/DATAVIZ_project/dataviz_venv')
print("files and directories:", files_and_directories)

: 

In [ ]:
files_and_directories = os.listdir('/home/rodrigo/repos/DATAVIZ_project/dataviz_venv/data')
print("files and directories:", files_and_directories)

: 

In [ ]:
df = pd.read_csv('/home/rodrigo/repos/DATAVIZ_project/dataviz_venv/data/blood_indicators.csv')
df.head()

: 

In [ ]:
#renaming missing columns
df.rename(columns={
    'lbxsldsi': 'Lactate Dehydrogenase (U/L)',
    'lbdsphsi': 'Phosphorus (mmol/L)',
    'lbxsksi': 'Potassium (mmol/L)',
    'lbxsgb': 'Globulin (g/dL)'
}, inplace=True)

df

: 

In [ ]:
# Healthy levels for young adults (age 20-39 years) - Reference ranges are approximate and may vary.
healthy_levels_young_adults = {
    "Albumin (g/dL)": {"min": 3.4, "max": 5.4},
    "Albumin (g/L)": {"min": 34, "max": 54},
    "Alanine aminotransferase ALT (U/L)": {"min": 5, "max": 40},
    "Aspartate aminotransferase AST (U/L)": {"min": 8, "max": 48},
    "Alkaline phosphatase (U/L)": {"min": 20, "max": 140},
    "Blood urea nitrogen (mg/dL)": {"min": 8, "max": 25},
    "Blood urea nitrogen (mmol/L)": {"min": 2.9, "max": 8.9},
    "Total calcium (mg/dL)": {"min": 8.6, "max": 10.4},
    "Total calcium (mmol/L)": {"min": 2.1, "max": 2.6},
    "Creatine Phosphokinase (CPK) (IU/L)": {"min": 38, "max": 308},
    "Cholesterol (mg/dL)": {"min": 125, "max": 200},
    "Cholesterol (mmol/L)": {"min": 3.24, "max": 5.18},
    "Bicarbonate (mmol/L)": {"min": 21, "max": 31},
    "Creatinine (mg/dL)": {"min": 0.7, "max": 1.3},
    "Creatinine (umol/L)": {"min": 61.9, "max": 115},
    "Gamma glutamyl transferase (U/L)": {"min": 5, "max": 85},
    "Glucose, serum (mg/dL)": {"min": 70, "max": 100},
    "Glucose, serum (mmol/L)": {"min": 3.9, "max": 5.6},
    "Iron, refrigerated (ug/dL)": {"min": 35, "max": 169},
    "Iron, refrigerated (umol/L)": {"min": 6.26, "max": 30.29},
    "Potassium (mmol/L)": {"min": 3.5, "max": 5.1},  
    "Phosphorus (mg/dL)": {"min": 2.5, "max": 4.9},
    "Phosphorus (mmol/L)": {"min": 0.81, "max": 1.58},  # Calculated from mg/dL
    "Total bilirubin (mg/dL)": {"min": 0.2, "max": 1.2},
    "Total bilirubin (umol/L)": {"min": 3.42, "max": 20.51},
    "Total protein (g/dL)": {"min": 6.6, "max": 8.3},
    "Total protein (g/L)": {"min": 66, "max": 83},
    "Uric acid (mg/dL)": {"min": 2.4, "max": 7.2},
    "Uric acid (umol/L)": {"min": 142, "max": 428},
    "Sodium (mmol/L)": {"min": 135, "max": 146},
    "Lactate Dehydrogenase (U/L)": {"min": 140, "max": 280},  
    "Chloride (mmol/L)": {"min": 96, "max": 106},
    "Osmolality (mmol/Kg)": {"min": 275, "max": 295},
    "Globulin (g/dL)": {"min": 2.3, "max": 3.5},  
    "Globulin (g/L)": {"min": 23, "max": 35},  
    "Triglycerides (mg/dL)": {"min": 30, "max": 150},
    "Triglycerides (mmol/L)": {"min": 0.34, "max": 1.69},
}



: 

In [ ]:
# Get the first row excluding the first two columns with only blood indictors levels
first_row = df.iloc[0][2:]
first_row

: 

In [ ]:
# transfroming the first row to dict
first_row_dict = first_row.to_dict()
first_row_dict

: 

In [ ]:
# function to get the correspondent color based on the reference range and where the level of the blood indicator sits
# red - outside normal range
# yellow - close to the edge
# green - in normal range

def get_blood_level_color(indicator_name, level, reference_ranges):
    if indicator_name in reference_ranges:
        reference = reference_ranges[indicator_name]
        min_range = reference.get("min")
        max_range = reference.get("max")

        if min_range is not None and max_range is not None:
            if level < min_range or level > max_range:
                return "red"
            elif abs(level - min_range) <= 0.2 * (max_range - min_range) or abs(level - max_range) <= 0.2 * (max_range - min_range):
                return "yellow"
            else:
                return "green"
        else:
            return "Unknown reference range"
    else:
        return "Indicator not found in the reference"

indicator_name = "Cholesterol (mg/dL)"
level = 160
color = get_blood_level_color(indicator_name, level, healthy_levels_young_adults)
print(f"The color for {indicator_name} at level {level} is {color}")


: 

In [ ]:
# fucntion to update the dict adding the correspondent color as a key value next to the blood level


def color_mapping(indicators_data, reference_ranges):
    colored_indicators = {}

    for indicator_name, level in indicators_data.items():
        colored_indicators[indicator_name] = [level, get_blood_level_color(indicator_name, level, reference_ranges)]
    return colored_indicators


def get_blood_level_color(indicator_name, level, reference_ranges):
    if indicator_name in reference_ranges:
        reference = reference_ranges[indicator_name]
        min_range = reference.get("min")
        max_range = reference.get("max")

        if min_range is not None and max_range is not None:
            if level < min_range or level > max_range:
                return "red"
            elif abs(level - min_range) <= 0.2 * (max_range - min_range) or abs(level - max_range) <= 0.2 * (
                    max_range - min_range):
                return "yellow"
            else:
                return "green"
        else:
            return "Unknown reference range"
    else:
        return "Indicator not found in the reference"


# Example usage
indicators_data = first_row_dict

colored_indicators = color_mapping(indicators_data, healthy_levels_young_adults)


colored_indicators
          
        

: 

In [ ]:
# heatmap using the updated colored_indicators dict which contains the correspondent color already

# Problem: the heatmap fucntion doesnt work, making the colors based on how high the value is, disrespecting different metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Your provided data
data = {'Albumin (g/dL)': [3.8, 'yellow'],
 'Albumin (g/L)': [38.0, 'yellow'],
 'Alanine aminotransferase ALT (U/L)': [14.0, 'green'],
 'Aspartate aminotransferase AST (U/L)': [14.0, 'yellow'],
 'Alkaline phosphatase (U/L)': [78.0, 'green'],
 'Blood urea nitrogen (mg/dL)': [32.0, 'red'],
 'Blood urea nitrogen (mmol/L)': [11.42, 'red'],
 'Total calcium (mg/dL)': [9.1, 'green'],
 'Total calcium (mmol/L)': [2.275, 'green'],
 'Creatine Phosphokinase (CPK) (IU/L)': [47.0, 'yellow'],
 'Cholesterol (mg/dL)': [86.0, 'red'],
 'Cholesterol (mmol/L)': [2.224, 'red'],
 'Bicarbonate (mmol/L)': [26.0, 'green'],
 'Creatinine (mg/dL)': [1.46, 'red'],
 'Creatinine (umol/L)': [129.06, 'red'],
 'Gamma glutamyl transferase (U/L)': [9.0, 'yellow'],
 'Glucose, serum (mg/dL)': [210.0, 'red'],
 'Glucose, serum (mmol/L)': [11.66, 'red'],
 'Iron, refrigerated (ug/dL)': [43.0, 'yellow'],
 'Iron, refrigerated (umol/L)': [7.7, 'yellow'],
 'Lactate Dehydrogenase (U/L)': [135.0, 'red'],
 'Phosphorus (mg/dL)': [4.9, 'yellow'],
 'Phosphorus (mmol/L)': [1.582, 'red'],
 'Total bilirubin (mg/dL)': [0.7, 'green'],
 'Total bilirubin (umol/L)': [11.97, 'green'],
 'Total protein (g/dL)': [7.2, 'green'],
 'Total protein (g/L)': [72.0, 'green'],
 'Uric acid (mg/dL)': [7.0, 'yellow'],
 'Uric acid (umol/L)': [416.4, 'yellow'],
 'Sodium (mmol/L)': [138.0, 'green'],
 'Potassium (mmol/L)': [4.3, 'green'],
 'Chloride (mmol/L)': [104.0, 'yellow'],
 'Osmolality (mmol/Kg)': [289.0, 'green'],
 'Globulin (g/dL)': [3.4, 'yellow'],
 'Globulin (g/L)': [34.0, 'yellow'],
 'Triglycerides (mg/dL)': [122.0, 'green'],
 'Triglycerides (mmol/L)': [1.377, 'green']
       }

# Create a DataFrame
df = pd.DataFrame(data.values(), index=data.keys(), columns=['Level', 'Color'])

# Define a custom colormap with three colors: green, yellow, and red
colors = ['#00FF00', '#FFFF00', '#FF0000']
cmap = ListedColormap(colors)

# Reshape the data for the heatmap using pivot_table
heatmap_data = df.pivot_table(index=df.index, columns="Color", values="Level", aggfunc='first', fill_value=np.nan)

# Create the heatmap using Matplotlib
fig, ax = plt.subplots()
cax = ax.matshow(heatmap_data, cmap=cmap, aspect='auto')
plt.colorbar(cax, fraction=0.03, pad=0.04)

# Set x-axis ticks and labels
ax.set_xticks(np.arange(len(heatmap_data.columns) + 1))  # Add 1 to the number of ticks
ax.set_xticklabels([''] + list(heatmap_data.columns))  # Add an empty label as the first one

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(heatmap_data.index)))
ax.set_yticklabels(heatmap_data.index)


plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.xlabel("Color/Level")
plt.ylabel("Blood Indicator")
plt.title("Blood Indicator Heatmap")

# Show values in the heatmap
for i in range(len(heatmap_data.index)):
    for j in range(len(heatmap_data.columns)):
        value = heatmap_data.iloc[i, j]
        if not np.isnan(value):
            ax.text(j, i, f'{value:.2f}', ha="center", va="center", color='black')

plt.tight_layout()
plt.show()



: 

In [ ]:

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 10))
cax = ax.matshow(df['Level'].values.reshape(1, -1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

plt.colorbar(cax, fraction=0.03, pad=0.04)

# Set x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

plt.ylabel("Blood Indicator")
plt.title("Blood Indicator Heatmap")

# Show values in the heatmap
for i, value in enumerate(df['Level']):
    ax.text(i, 0, f'{value:.2f}', ha="center", va="center", color='black')

plt.tight_layout()
plt.show()


: 

In [ ]:
df

: 

In [ ]:
# colors correct but levels missing

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    level = values['Level']
    ax.text(0.5, i, f'{indicator}\n{level:.2f}', ha="center", va="center", color=color)

plt.title("Blood Indicator Heatmap")
plt.show()

: 

In [ ]:

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5))

plt.title("Blood Indicator Heatmap")
plt.show()

: 

In [ ]:
# removed wrong numbers on top in the x axis

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5))

plt.title("Blood Indicator Heatmap")
plt.show()

: 

In [ ]:
# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Hide colorbar ticks and labels
cbar = plt.colorbar(cax, fraction=0.03, pad=0.04)
cbar.set_ticks([])

# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5))

plt.title("Blood Indicator Heatmap")
plt.show()

: 

In [ ]:
# added black line for better readability

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])


# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5, linewidth=0.5))

    # Add vertical line to separate each y label
    ax.axhline(i, color='black', linewidth=0.5)

plt.title("Blood Indicator Heatmap")
plt.show()

: 

In [ ]:

from matplotlib.colors import ListedColormap


# Assuming df is your DataFrame

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Show blood indicator levels in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    # Display blood indicator levels on the right side of the heatmap
    ax.text(1.5, i, f'{values["Level"]:.3f}', ha="center", va="center", color='black')

    # Display color-coded indicators on the left side of the heatmap
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5))

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Add a title
plt.title("Blood Indicator Heatmap with Levels")
plt.show()



: 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

# Assuming df is your DataFrame

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
heatmap = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Show blood indicator levels in the center of x label
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (heatmap.get_clim()[0] + heatmap.get_clim()[1]) / 2
    
    # Display rounded blood indicator levels in the center of x label
    ax.text(0.5, i, f'{round(values["Level"], 3)}', ha="left", va="center", color='black')

    # Add vertical line to separate each y label
    ax.axhline(i, color='black', linewidth=0.5)

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Add a title
plt.title("Blood Indicator Heatmap")
plt.show()




: 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd

# Assuming df is your DataFrame

# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
heatmap = ax.matshow(df['Level'].values.reshape(-1, 1), cmap=ListedColormap(df['Color'].tolist()), aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Show blood indicator levels in the center of x label
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (heatmap.get_clim()[0] + heatmap.get_clim()[1]) / 2
    
    # Display blood indicator levels in the center of x label
    ax.text(0.5, i, f'{values["Level"]:.3f}', ha="left", va="center", color='black')

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Add a title
plt.title("Blood Indicator Heatmap")
plt.show()



: 

In [ ]:
# Calculate the percentage of each color
color_count = df['Color'].value_counts()
percentage = (color_count / len(df)) * 100
percentage = dict(percentage)
percentage

: 

In [ ]:

colors = ['#00FF00', '#FFFF00', '#FF0000']

# Plotting the pie chart
fig, ax = plt.subplots()
ax.pie(percentage.values(), labels=percentage.keys(), autopct='%1.1f%%', startangle=90, colors=colors)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Adding a title
plt.title('Color Distribution')

# Display the pie chart
plt.show()

: 

In [ ]:


colors = ['#006400', '#FFFF00', '#FF0000']

# Plotting the pie chart
fig, ax = plt.subplots()
ax.pie(percentage.values(), labels=percentage.keys(), autopct='%1.1f%%', startangle=90, colors=colors)
ax.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Adding a title
plt.title('Color Distribution')

# Display the pie chart
plt.show()



: 

In [ ]:
# trying to implement gradient visualization but visualization gets worse

from matplotlib.colors import LinearSegmentedColormap



# Create the heatmap using Matplotlib
fig, ax = plt.subplots(figsize=(8, 12))
cax = ax.matshow(df['Level'].values.reshape(-1, 1), cmap='RdYlGn', aspect='auto')

# Set y-axis ticks and labels
ax.set_yticks(np.arange(len(df)))
ax.set_yticklabels(df.index)

# Hide x-axis ticks and labels
ax.set_xticks([])
ax.set_xticklabels([])

# Create a custom colormap with three colors: green, yellow, and red
colors = ['#00FF00', '#FFFF00', '#FF0000']
cmap = LinearSegmentedColormap.from_list('Custom', colors, N=256)

# Customize colorbar
cbar = plt.colorbar(cax, fraction=0.04, pad=0.04, cmap=cmap, ticks=[0, 1])
cbar.set_label('Color Range')  # Set label for the colorbar

# Show values in the heatmap
for i, (indicator, values) in enumerate(df.iterrows()):
    color = values['Color']
    
    # Get the middle of the color display
    color_display_middle = (cax.get_clim()[0] + cax.get_clim()[1]) / 2
    
    ax.text(0.5, i, '', ha="center", va="center", color='black', bbox=dict(facecolor=color, edgecolor='black', boxstyle='round', pad=0.5, linewidth=0.5))

    # Add vertical line to separate each y label
    ax.axhline(i, color='black', linewidth=0.5)

plt.title("Blood Indicator Heatmap")
plt.show()





: 

: 

In [ ]:
# fucntion to tranform into FHIR format


import json
from datetime import datetime

def transform_to_fhir_blood_test(data, patient_reference="Patient/12345"):
    fhir_observation = {
        "resourceType": "Observation",
        "status": "final",
        "category": [
            {
                "coding": [
                    {
                        "system": "http://terminology.hl7.org/CodeSystem/observation-category",
                        "code": "laboratory",
                        "display": "Laboratory"
                    }
                ],
                "text": "Laboratory"
            }
        ],
        "code": {
            "coding": [
                {
                    "system": "http://loinc.org",
                    "code": "24357-5",
                    "display": "Blood Panel"
                }
            ],
            "text": "Blood Panel"
        },
        "subject": {
            "reference": patient_reference
        },
        "effectiveDateTime": datetime.now().isoformat(),
        "component": []
    }

    for test_name, (result_value, color) in data.items():
        unit, unit_code = determine_unit_and_code(test_name)
        interpretation_code = determine_interpretation_code(color)
        component = {
            "code": {
                "text": test_name
            },
            "valueQuantity": {
                "value": result_value,
                "unit": unit,
                "system": "http://unitsofmeasure.org",
                "code": unit_code
            },
            "interpretation": {
                "coding": [
                    {
                        "system": "http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation",
                        "code": interpretation_code,
                        "display": interpretation_code.capitalize()
                    }
                ]
            }
        }
        fhir_observation["component"].append(component)

    return json.dumps(fhir_observation, indent=2)

def determine_unit_and_code(test_name):
    # Add logic to determine the unit and unit code based on the test name
    # You might need more sophisticated logic or external reference data
    if "mg/dL" in test_name:
        return "mg/dL", "mg/dL"
    elif "g/dL" in test_name:
        return "g/dL", "g/dL"
    elif "U/L" in test_name:
        return "U/L", "U/L"
    elif "mmol/L" in test_name:
        return "mmol/L", "mmol/L"
    elif "ug/dL" in test_name:
        return "ug/dL", "ug/dL"
    elif "umol/L" in test_name:
        return "umol/L", "umol/L"
    else:
        return "unknown", "unknown"

def determine_interpretation_code(color):
    # Map color to interpretation code based on the provided mapping
    if color.lower() == "green":
        return "N"  # Normal range
    elif color.lower() == "yellow":
        return "R"  # Medium range
    elif color.lower() == "red":
        return "H"  # Extreme range
    else:
        return "unknown"

# Example usage:
blood_test_data = {
    "Albumin (g/dL)": [3.8, "yellow"],
    "Albumin (g/L)": [38.0, "yellow"],
    "Alanine aminotransferase ALT (U/L)": [14.0, "green"],
    "Aspartate aminotransferase AST (U/L)": [14.0, "yellow"],
    "Alkaline phosphatase (U/L)": [78.0, "green"],
    "Blood urea nitrogen (mg/dL)": [32.0, "red"],
    "Blood urea nitrogen (mmol/L)": [11.42, "red"],
    "Total calcium (mg/dL)": [9.1, "green"],
    "Total calcium (mmol/L)": [2.275, "green"],
    "Creatine Phosphokinase (CPK) (IU/L)": [47.0, "Unknown reference range"],
    "Cholesterol (mg/dL)": [86.0, "red"],
    "Cholesterol (mmol/L)": [2.224, "red"],
    "Bicarbonate (mmol/L)": [26.0, "green"],
    "Creatinine (mg/dL)": [1.46, "red"],
    "Creatinine (umol/L)": [129.06, "red"],
    "Gamma glutamyl transferase (U/L)": [9.0, "yellow"],
    "Glucose, serum (mg/dL)": [210.0, "red"],
    "Glucose, serum (mmol/L)": [11.66, "red"],
    "Iron, refrigerated (ug/dL)": [43.0, "yellow"],
    "Iron, refrigerated (umol/L)": [7.7, "yellow"],
    "lbxsldsi": [135.0, "Unknown reference range"],
    "Phosphorus (mg/dL)": [4.9, "yellow"],
    "lbdsphsi": [1.582, "Unknown reference range"],
    "Total bilirubin (mg/dL)": [0.7, "green"],
    "Total bilirubin (umol/L)": [11.97, "green"],
    "Total protein (g/dL)": [7.2, "green"],
    "Total protein (g/L)": [72.0, "green"],
    "Uric acid (mg/dL)": [7.0, "yellow"],
    "Uric acid (umol/L)": [416.4, "yellow"],
    "Sodium (mmol/L)": [138.0, "green"],
    "lbxsksi": [4.3, "Unknown reference range"],
    "Chloride (mmol/L)": [104.0, "yellow"],
    "Osmolality (mmol/Kg)": [289.0, "green"],
    "lbxsgb": [3.4, "Unknown reference range"],
    "Globulin (g/L)": [34.0, "yellow"],
    "Triglycerides (mg/dL)": [122.0, "green"],
    "Triglycerides (mmol/L)": [1.377, "green"]
}

fhir_observation_json = transform_to_fhir_blood_test(blood_test_data)
print(fhir_observation_json)


: 

: 